In [1]:
import pickle
import numpy as np
import rowan
from src.quaternions import dr_orientation_angles
import pandas as pd

In [2]:
with open('ellipsoid_arrangment_logs.json', 'rb') as f:
    special_cases_log = pickle.load(f)

### Proxy #1: angles between dr and each particle's orientation z-axis

In [38]:
angle_proxy = pd.DataFrame(columns=['energy', 
                                    # 'force_x','force_y', 'force_z',
                                    'theta', 'phi'])
for i, (case, v) in enumerate(special_cases_log.items()):
    theta = dr_orientation_angles(v['dr'], v['q1'], degree=True)
    phi = dr_orientation_angles(v['dr'], v['q2'], degree=True)
    angle_proxy.loc[i] = {'energy': v['energy'], 
                        # 'force_x':  v['forces'][0][0], 
                        # 'force_y': v['forces'][0][1],
                        # 'force_z': v['forces'][0][2],
                        'theta': np.around(np.array(theta), 2), 
                        'phi': np.around(np.array(phi), 2)}

In [39]:

angle_proxy.sort_values('energy')

,energy,theta,phi
12,-3.54,"[90.0, 90.0, 0.0]","[90.0, 0.0, 90.0]"
11,-3.08,"[90.0, 90.0, 180.0]","[90.0, 0.0, 90.0]"
1,-2.21,"[180.0, 90.0, 90.0]","[180.0, 90.0, 90.0]"
0,-2.07,"[180.0, 90.0, 90.0]","[180.0, 90.0, 90.0]"
5,-1.69,"[180.0, 90.0, 90.0]","[90.0, 0.0, 90.0]"
4,-1.67,"[180.0, 90.0, 90.0]","[90.0, 0.0, 90.0]"
3,-1.19,"[90.0, 0.0, 90.0]","[90.0, 0.0, 90.0]"
2,-1.19,"[90.0, 0.0, 90.0]","[90.0, 0.0, 90.0]"
9,24.29,"[90.0, 90.0, 180.0]","[180.0, 90.0, 90.0]"
10,42.16,"[90.0, 90.0, 0.0]","[180.0, 90.0, 90.0]"


Comment: theta and phi angles don't seem to be sufficient to represent huge the differences in force between different cases. For example, 12 and 10 have very different energies but the proxy angles are similar.

### Proxy #2: dot prod of dr and each particle's orientation z-axis

In [30]:
dot_proxy = pd.DataFrame(columns=['energy', 'force_x','force_y', 'force_z', 'dr.q1', 'dr.q2'])
for i, (case, v) in enumerate(special_cases_log.items()):
    dot1 = np.dot(v['dr'], rowan.to_matrix(v['q1'])[:, 2])
    dot2 = np.dot(v['dr'], rowan.to_matrix(v['q2'])[:, 2])
    dot_proxy.loc[i] = {'energy': v['energy'],
                        'force_x':  v['forces'][0][0], 
                        'force_y': v['forces'][0][1],
                        'force_z': v['forces'][0][2],
                        'dr.q1': dot1,
                        'dr.q2': dot2}

In [31]:
dot_proxy.sort_values('energy')

,energy,force_x,force_y,force_z,dr.q1,dr.q2
12,-3.538435,5.756357e+00,3.230676e-06,2.533216e-01,1.7,0.0
11,-3.082100,6.790192e+00,1.330837e-06,1.125634e-01,-1.7,0.0
1,-2.208676,4.959775e+00,9.048788e-04,-2.160355e-02,0.0,0.0
0,-2.070525,1.298373e+00,-2.141217e-06,-6.309236e-08,0.0,0.0
5,-1.687354,5.471506e+00,1.525854e-02,-6.549757e-02,0.0,0.0
4,-1.667615,5.419017e+00,7.642513e-03,-2.412387e-01,0.0,0.0
3,-1.189672,3.645067e+00,-1.671528e-06,1.875642e-02,0.0,0.0
2,-1.189198,3.646761e+00,-9.358272e-07,-7.343452e-02,0.0,0.0
9,24.287839,-4.832682e+02,2.328814e+00,-8.079457e+01,-1.7,0.0
10,42.158973,-7.852775e+02,3.980410e+00,-1.289475e+02,1.7,0.0


same comment for dot product.

### proxy #3: 